In [1]:
import breizhcrops as bzh
from breizhcrops import BreizhCrops
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import Adam
import os
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, recall_score, precision_score
import sklearn

In [2]:
frh01 = BreizhCrops("frh01")
frh02 = BreizhCrops("frh02")
frh03 = BreizhCrops("frh03")
frh04 = BreizhCrops("frh04")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
frh01.domain = 1
frh02.domain = 2
frh03.domain = 3
frh04.domain = 4

In [4]:
def split_dataset(dataset, X=0.8):

    total_samples = len(dataset)

    num_test_samples = int(X * total_samples)

    np.random.seed(42)

    test_indices = np.random.choice(total_samples, num_test_samples, replace=False)

    # Find complementary indices
    train_indices = np.setdiff1d(np.arange(total_samples), test_indices)

    test = torch.utils.data.Subset(dataset, test_indices)
    train = torch.utils.data.Subset(dataset, train_indices)
    return train, test

In [5]:
def get_dataloader(mode, batchsize, X=0.8, num_workers=0, upperPerformance=False):


    if mode == "234":
        frh01_train, frh01_test = split_dataset(frh01, X=X)

        if upperPerformance:
            traindatasets = frh01_train
        else:
            traindatasets = torch.utils.data.ConcatDataset([frh02, frh03, frh04])

        testdataset = frh01_test

    elif mode =='123':
        frh04_train, frh04_test = split_dataset(frh04, X=X)

        if upperPerformance:
            traindatasets = frh04_train
        else:
            traindatasets = torch.utils.data.ConcatDataset([frh01, frh02, frh03])

        testdataset = frh04_test

    elif mode == '124':
        frh03_train, frh03_test = split_dataset(frh03, X=X)

        if upperPerformance:
            traindatasets = frh03_train
        else:
            traindatasets = torch.utils.data.ConcatDataset([frh01, frh02, frh04])

        testdataset = frh03_test
    elif mode == '134':
        frh02_train, frh02_test = split_dataset(frh02, X=X)

        if upperPerformance:
            traindatasets = frh02_train
        else:
            traindatasets = torch.utils.data.ConcatDataset([frh01, frh03, frh04])

        testdataset = frh02_test
    else:
        raise ValueError("Invalid mode")


    traindataloader = DataLoader(traindatasets, batch_size=batchsize, shuffle=True, num_workers=num_workers)
    testdataloader = DataLoader(testdataset, batch_size=batchsize, shuffle=False, num_workers=num_workers)

    meta = dict(
        ndims=13,
        num_classes=frh02.classes,
        sequencelength=45
    )

    return traindataloader, testdataloader, meta

In [6]:
class LSTM_Model(torch.nn.Module):
    def __init__(self, input_size=13, hidden_size=128, num_layers=4, output_size=9, bidirectional=False, dropout=0.2, c_norm=True):
        super(LSTM_Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.c_norm = c_norm

        if num_layers > 1 and dropout > 0:
            self.dropout = dropout
        else:
            self.dropout = 0.0

        self.clayernorm = nn.LayerNorm((hidden_size + hidden_size * bidirectional) * num_layers)

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
                            bias=False, batch_first=True, dropout=dropout, bidirectional=bidirectional)

        if bidirectional:
            hidden_size = hidden_size * 2

        self.fc = nn.Linear(hidden_size * num_layers, output_size, bias=True)

    def forward(self, x):

        outputs, last_state_list = self.lstm.forward(x)

        h, c = last_state_list

        nlayers, batchsize, n_hidden = c.shape

        if self.c_norm:
            x = self.clayernorm(c.transpose(0, 1).contiguous().view(batchsize, nlayers * n_hidden))
        else:
            x = c.transpose(0, 1).contiguous().view(batchsize, nlayers * n_hidden)
        x = self.fc.forward(x)
        return x

    def save(self, path="model.pth", **kwargs):
        print("\nsaving model to " + path)
        model_state = self.state_dict()
        os.makedirs(os.path.dirname(path), exist_ok=True)
        torch.save(dict(model_state=model_state, **kwargs), path)

model = LSTM_Model(input_size=13, hidden_size=128, num_layers=4, output_size=9, bidirectional=False, dropout=0.2, c_norm=True).to(device)


In [12]:
def metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    f1_micro = f1_score(y_true, y_pred, average="micro")
    f1_macro = f1_score(y_true, y_pred, average="macro")
    f1_weighted = f1_score(y_true, y_pred, average="weighted")
    recall_micro = recall_score(y_true, y_pred, average="micro")
    recall_macro = recall_score(y_true, y_pred, average="macro")
    recall_weighted = recall_score(y_true, y_pred, average="weighted")
    precision_micro = precision_score(y_true, y_pred, average="micro")
    precision_macro = precision_score(y_true, y_pred, average="macro")
    precision_weighted = precision_score(y_true, y_pred, average="weighted")

    return dict(
        accuracy=accuracy,
        kappa=kappa,
        f1_micro=f1_micro,
        f1_macro=f1_macro,
        f1_weighted=f1_weighted,
        recall_micro=recall_micro,
        recall_macro=recall_macro,
        recall_weighted=recall_weighted,
        precision_micro=precision_micro,
        precision_macro=precision_macro,
        precision_weighted=precision_weighted,
    )


def train_epoch(model, optimizer, criterion, dataloader, device):
    model.train()
    losses = list()
    with tqdm(enumerate(dataloader), total=len(dataloader), leave=True) as iterator:
        for idx, batch in iterator:
            optimizer.zero_grad()
            x, y_true, _, _= batch
            loss = criterion(model.forward(x.to(device)), y_true.to(device))
            loss.backward()
            optimizer.step()
            iterator.set_description(f"train loss={loss:.2f}")
            losses.append(loss)
    return torch.stack(losses)


def test_epoch(model, criterion, dataloader, device):
    model.eval()
    with torch.no_grad():
        losses = list()
        y_true_list = list()
        y_pred_list = list()
        field_ids_list = list()
        with tqdm(enumerate(dataloader), total=len(dataloader), leave=True) as iterator:
            for idx, batch in iterator:
                x, y_true, field_id, _ = batch
                logits = model.forward(x.to(device))
                loss = criterion(logits, y_true.to(device))
                iterator.set_description(f"test loss={loss:.2f}")
                losses.append(loss)
                y_true_list.append(y_true)
                y_pred_list.append(logits.argmax(-1))
                field_ids_list.append(field_id)
        return torch.stack(losses), torch.cat(y_true_list), torch.cat(y_pred_list), torch.cat(field_ids_list)


# Lower Baseline

## 123

In [13]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = LSTM_Model(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True).to(device)
train_dataloader, test_dataloader, meta = get_dataloader(mode="123", batchsize=32, num_workers=0, X=0.2, upperPerformance=False)
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.005, weight_decay=1e-6)
log = list()
for epoch in range(10):
    train_loss = train_epoch(model, optimizer, criterion, train_dataloader, device)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy()[0]
    train_loss = train_loss.cpu().detach().numpy()[0]
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    log.append(scores1)

    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("Lower123.csv")

  0%|          | 0/15177 [00:00<?, ?it/s]

test loss=1.90: 100%|██████████| 767/767 [00:35<00:00, 21.57it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0: trainloss 2.39, testloss 0.75 accuracy=0.75, kappa=0.68, f1_micro=0.75, f1_macro=0.50, f1_weighted=0.72, recall_micro=0.75, recall_macro=0.52, recall_weighted=0.75, precision_micro=0.75, precision_macro=0.52, precision_weighted=0.75


test loss=1.71: 100%|██████████| 767/767 [00:35<00:00, 21.73it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1: trainloss 0.35, testloss 0.82 accuracy=0.77, kappa=0.69, f1_micro=0.77, f1_macro=0.53, f1_weighted=0.75, recall_micro=0.77, recall_macro=0.53, recall_weighted=0.77, precision_micro=0.77, precision_macro=0.55, precision_weighted=0.76


...

epoch 7: trainloss 0.54, testloss 0.75 accuracy=0.79, kappa=0.72, f1_micro=0.79, f1_macro=0.56, f1_weighted=0.79, recall_micro=0.79, recall_macro=0.56, recall_weighted=0.79, precision_micro=0.79, precision_macro=0.56, precision_weighted=0.78


test loss=0.72: 100%|██████████| 767/767 [00:31<00:00, 24.28it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 8: trainloss 0.28, testloss 0.83 accuracy=0.79, kappa=0.73, f1_micro=0.79, f1_macro=0.56, f1_weighted=0.78, recall_micro=0.79, recall_macro=0.56, recall_weighted=0.79, precision_micro=0.79, precision_macro=0.56, precision_weighted=0.79


test loss=1.30: 100%|██████████| 767/767 [00:29<00:00, 26.29it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 9: trainloss 0.39, testloss 0.75 accuracy=0.78, kappa=0.71, f1_micro=0.78, f1_macro=0.53, f1_weighted=0.76, recall_micro=0.78, recall_macro=0.55, recall_weighted=0.78, precision_micro=0.78, precision_macro=0.53, precision_weighted=0.77


In [14]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = LSTM_Model(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True).to(device)
train_dataloader, test_dataloader, meta = get_dataloader(mode="124", batchsize=32, num_workers=0, X=0.2, upperPerformance=False)
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.005, weight_decay=1e-6)
log = list()
for epoch in range(10):
    train_loss = train_epoch(model, optimizer, criterion, train_dataloader, device)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy()[0]
    train_loss = train_loss.cpu().detach().numpy()[0]
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    log.append(scores1)

    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("Lower124.csv")

test loss=1.76: 100%|██████████| 1040/1040 [00:56<00:00, 18.39it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0: trainloss 2.42, testloss 0.77 accuracy=0.75, kappa=0.67, f1_micro=0.75, f1_macro=0.51, f1_weighted=0.74, recall_micro=0.75, recall_macro=0.51, recall_weighted=0.75, precision_micro=0.75, precision_macro=0.51, precision_weighted=0.74


test loss=0.82: 100%|██████████| 1040/1040 [01:00<00:00, 17.32it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1: trainloss 0.46, testloss 0.75 accuracy=0.74, kappa=0.66, f1_micro=0.74, f1_macro=0.49, f1_weighted=0.73, recall_micro=0.74, recall_macro=0.52, recall_weighted=0.74, precision_micro=0.74, precision_macro=0.49, precision_weighted=0.74


test loss=0.86: 100%|██████████| 1040/1040 [00:58<00:00, 17.66it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

epoch 7: trainloss 0.43, testloss 0.71 accuracy=0.76, kappa=0.68, f1_micro=0.76, f1_macro=0.50, f1_weighted=0.71, recall_micro=0.76, recall_macro=0.53, recall_weighted=0.76, precision_micro=0.76, precision_macro=0.52, precision_weighted=0.75


test loss=0.82: 100%|██████████| 1040/1040 [01:00<00:00, 17.20it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 8: trainloss 0.41, testloss 0.81 accuracy=0.77, kappa=0.70, f1_micro=0.77, f1_macro=0.53, f1_weighted=0.76, recall_micro=0.77, recall_macro=0.55, recall_weighted=0.77, precision_micro=0.77, precision_macro=0.52, precision_weighted=0.76


test loss=0.80: 100%|██████████| 1040/1040 [01:03<00:00, 16.49it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 9: trainloss 0.55, testloss 0.71 accuracy=0.78, kappa=0.71, f1_micro=0.78, f1_macro=0.55, f1_weighted=0.77, recall_micro=0.78, recall_macro=0.55, recall_weighted=0.78, precision_micro=0.78, precision_macro=0.55, precision_weighted=0.77


In [15]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = LSTM_Model(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True).to(device)
train_dataloader, test_dataloader, meta = get_dataloader(mode="134", batchsize=32, num_workers=0, X=0.2, upperPerformance=False)
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.005, weight_decay=1e-6)
log = list()
for epoch in range(10):
    train_loss = train_epoch(model, optimizer, criterion, train_dataloader, device)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy()[0]
    train_loss = train_loss.cpu().detach().numpy()[0]
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    log.append(scores1)

    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("Lower134.csv")

test loss=0.69: 100%|██████████| 880/880 [00:53<00:00, 16.55it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0: trainloss 2.15, testloss 0.58 accuracy=0.76, kappa=0.68, f1_micro=0.76, f1_macro=0.53, f1_weighted=0.75, recall_micro=0.76, recall_macro=0.53, recall_weighted=0.76, precision_micro=0.76, precision_macro=0.54, precision_weighted=0.76


test loss=0.55: 100%|██████████| 880/880 [00:50<00:00, 17.34it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1: trainloss 0.52, testloss 0.46 accuracy=0.75, kappa=0.67, f1_micro=0.75, f1_macro=0.53, f1_weighted=0.73, recall_micro=0.75, recall_macro=0.54, recall_weighted=0.75, precision_micro=0.75, precision_macro=0.55, precision_weighted=0.78


test loss=0.57: 100%|██████████| 880/880 [00:49<00:00, 17.68it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

epoch 7: trainloss 0.60, testloss 0.40 accuracy=0.77, kappa=0.70, f1_micro=0.77, f1_macro=0.55, f1_weighted=0.77, recall_micro=0.77, recall_macro=0.54, recall_weighted=0.77, precision_micro=0.77, precision_macro=0.57, precision_weighted=0.79


test loss=0.46: 100%|██████████| 880/880 [00:50<00:00, 17.49it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 8: trainloss 0.71, testloss 0.47 accuracy=0.75, kappa=0.68, f1_micro=0.75, f1_macro=0.54, f1_weighted=0.73, recall_micro=0.75, recall_macro=0.54, recall_weighted=0.75, precision_micro=0.75, precision_macro=0.56, precision_weighted=0.79


test loss=0.71: 100%|██████████| 880/880 [00:50<00:00, 17.50it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 9: trainloss 0.38, testloss 0.40 accuracy=0.77, kappa=0.71, f1_micro=0.77, f1_macro=0.55, f1_weighted=0.77, recall_micro=0.77, recall_macro=0.54, recall_weighted=0.77, precision_micro=0.77, precision_macro=0.56, precision_weighted=0.78


# Upper Baseline

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTM_Model(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True).to(device)
train_dataloader, test_dataloader, meta = get_dataloader(mode="123", batchsize=32, num_workers=0, X=0.2, upperPerformance=True)
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.005, weight_decay=1e-6)
log = list()
for epoch in range(10):
    train_loss = train_epoch(model, optimizer, criterion, train_dataloader, device)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy()[0]
    train_loss = train_loss.cpu().detach().numpy()[0]
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    log.append(scores1)

    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("Upper123.csv")

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTM_Model(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True).to(device)
train_dataloader, test_dataloader, meta = get_dataloader(mode="124", batchsize=32, num_workers=0, X=0.2, upperPerformance=True)
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.005, weight_decay=1e-6)
log = list()
for epoch in range(10):
    train_loss = train_epoch(model, optimizer, criterion, train_dataloader, device)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy()[0]
    train_loss = train_loss.cpu().detach().numpy()[0]
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    log.append(scores1)

    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("Upper124.csv")

test loss=0.69: 100%|██████████| 1040/1040 [01:07<00:00, 15.38it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0: trainloss 2.01, testloss 0.84 accuracy=0.67, kappa=0.57, f1_micro=0.67, f1_macro=0.37, f1_weighted=0.64, recall_micro=0.67, recall_macro=0.37, recall_weighted=0.67, precision_micro=0.67, precision_macro=0.38, precision_weighted=0.65


test loss=0.97: 100%|██████████| 1040/1040 [01:03<00:00, 16.30it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1: trainloss 0.72, testloss 0.74 accuracy=0.75, kappa=0.67, f1_micro=0.75, f1_macro=0.50, f1_weighted=0.74, recall_micro=0.75, recall_macro=0.50, recall_weighted=0.75, precision_micro=0.75, precision_macro=0.51, precision_weighted=0.74


test loss=0.90: 100%|██████████| 1040/1040 [01:06<00:00, 15.53it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

epoch 7: trainloss 0.53, testloss 0.78 accuracy=0.77, kappa=0.69, f1_micro=0.77, f1_macro=0.50, f1_weighted=0.75, recall_micro=0.77, recall_macro=0.52, recall_weighted=0.77, precision_micro=0.77, precision_macro=0.51, precision_weighted=0.76


test loss=0.78: 100%|██████████| 1040/1040 [00:53<00:00, 19.33it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 8: trainloss 0.51, testloss 0.75 accuracy=0.78, kappa=0.72, f1_micro=0.78, f1_macro=0.54, f1_weighted=0.78, recall_micro=0.78, recall_macro=0.54, recall_weighted=0.78, precision_micro=0.78, precision_macro=0.55, precision_weighted=0.77


test loss=0.70: 100%|██████████| 1040/1040 [01:05<00:00, 15.90it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 9: trainloss 0.54, testloss 0.77 accuracy=0.77, kappa=0.70, f1_micro=0.77, f1_macro=0.53, f1_weighted=0.76, recall_micro=0.77, recall_macro=0.52, recall_weighted=0.77, precision_micro=0.77, precision_macro=0.56, precision_weighted=0.77


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTM_Model(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True).to(device)
train_dataloader, test_dataloader, meta = get_dataloader(mode="134", batchsize=32, num_workers=0, X=0.2, upperPerformance=True)
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.005, weight_decay=1e-6)
log = list()
for epoch in range(10):
    train_loss = train_epoch(model, optimizer, criterion, train_dataloader, device)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy()[0]
    train_loss = train_loss.cpu().detach().numpy()[0]
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    log.append(scores1)

    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("Upper134.csv")

test loss=0.57: 100%|██████████| 880/880 [01:00<00:00, 14.59it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0: trainloss 2.45, testloss 0.51 accuracy=0.67, kappa=0.57, f1_micro=0.67, f1_macro=0.42, f1_weighted=0.67, recall_micro=0.67, recall_macro=0.41, recall_weighted=0.67, precision_micro=0.67, precision_macro=0.45, precision_weighted=0.67


test loss=0.57: 100%|██████████| 880/880 [00:56<00:00, 15.64it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1: trainloss 0.67, testloss 0.58 accuracy=0.72, kappa=0.64, f1_micro=0.72, f1_macro=0.48, f1_weighted=0.72, recall_micro=0.72, recall_macro=0.49, recall_weighted=0.72, precision_micro=0.72, precision_macro=0.48, precision_weighted=0.72


test loss=0.44: 100%|██████████| 880/880 [00:59<00:00, 14.85it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

epoch 7: trainloss 0.52, testloss 0.51 accuracy=0.78, kappa=0.71, f1_micro=0.78, f1_macro=0.55, f1_weighted=0.77, recall_micro=0.78, recall_macro=0.55, recall_weighted=0.78, precision_micro=0.78, precision_macro=0.55, precision_weighted=0.77


test loss=0.34: 100%|██████████| 880/880 [00:58<00:00, 15.05it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 8: trainloss 0.58, testloss 0.40 accuracy=0.77, kappa=0.71, f1_micro=0.77, f1_macro=0.55, f1_weighted=0.77, recall_micro=0.77, recall_macro=0.55, recall_weighted=0.77, precision_micro=0.77, precision_macro=0.54, precision_weighted=0.77


test loss=0.29: 100%|██████████| 880/880 [00:57<00:00, 15.21it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 9: trainloss 0.42, testloss 0.49 accuracy=0.77, kappa=0.70, f1_micro=0.77, f1_macro=0.55, f1_weighted=0.77, recall_micro=0.77, recall_macro=0.55, recall_weighted=0.77, precision_micro=0.77, precision_macro=0.55, precision_weighted=0.78
